In [9]:
import mlflow
import os
import logging
import sagemaker


from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA
import sagemaker
import boto3
import json
import subprocess
from langchain_aws import ChatBedrockConverse
from os import environ
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import OpenSearchVectorSearch
from opensearchpy import RequestsHttpConnection, AWSV4SignerAuth
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from typing import List
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

os.environ['MLFLOW_TRACKING_URI']="http://localhost:8080"

model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
encode_kwargs = {'normalize_embeddings': False}

class CustomEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return [self.model.encode(d).tolist() for d in documents]

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0].tolist()
    
model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

embeddings = CustomEmbeddings(model_name=model_name)

def get_opensearch():
    docsearch = OpenSearchVectorSearch(
        index_name="aoss-index",  # TODO: use the same index-name used in the ingestion script
        embedding_function=embeddings,
        opensearch_url="https://epcavlvwitam2ivpwv4k.eu-west-2.aoss.amazonaws.com:443",  # TODO: e.g. use the AWS OpenSearch domain instantiated previously
        http_auth = auth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection,
    )

    return docsearch

def format_docs(docs):
    context = []
    for d in docs:
        program_description = ""
        if d.metadata["mediacontent_page_description_program"]  != "":
            program_description = docs[0].metadata["mediacontent_page_description_program"]
        elif d.metadata["mediacontent_page_editorialtitle_program"]  != "":
            program_description = docs[0].metadata["mediacontent_page_editorialtitle_program"]

        context.append("Het programma met de naam " + d.metadata["mediacontent_pagetitle_program"] +\
                " heeft volgende beschrijving: " + program_description  +\
                " De episode van dit programma heeft als beschrijving: " + d.metadata["mediacontent_page_description"] +\
                " De episode zal online staan tot " + d.metadata["offering_publication_planneduntil"] +\
                " De episode heeft als URL " + d.metadata["mediacontent_pageurl"] +\
                " De episode heeft als foto " + "https:" +d.metadata["mediacontent_imageurl"] 
        )
        
    return "\n\n".join(context)

class RAGModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, question):

        # LLM
        llm = ChatBedrockConverse(
            region_name="eu-west-2",
            model="mistral.mistral-large-2402-v1:0" , # "meta.llama3-70b-instruct-v1:0"
            temperature=0.6,
            top_p=0.6,
            max_tokens=512
        )

        # VECTOR DB
        docsearch = get_opensearch()
        retriever = docsearch.as_retriever(search_kwargs={'k': 5,"vector_field":"vrtmax_catalog_vector"})

        # PROMPTING
        system_prompt = """
        Je bent een hulpvaardige assistent die Nederlands spreekt.
        Je krijgt vragen over de catalogus van programma's van het videoplatform VRT MAX. 
        Je probeert mensen te helpen om programma's aan te bevelen die aansluiten bij hun vraag.
        Daarvoor krijg je een beschrijving van een aantal programma's.
        Aan jou om wat relevant is aan te bevelen.
        Begin niet met het geven van jouw mening over de programma's. Begin direct met het aanbevelen van relevante content aan de gebruiker.
        """

        message = """Beantwoordt de vraag op basis van de volgende context:

        {context}

        Vraag: {question}
        """
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", message),
            ]
        )

        
        from langchain_core.runnables import RunnableParallel

        rag_chain_from_docs = (
            RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
            | prompt
            | llm
            | StrOutputParser()
        )

        rag_chain_with_source = RunnableParallel(
            {"context": retriever, "question": RunnablePassthrough()}
        ).assign(answer=rag_chain_from_docs)

        config = {"configurable": {"session_id": "special_key"}}

        response = rag_chain_with_source.invoke({"question": question}, config)

        return response

model = RAGModel()
os.environ['MLFLOW_S3_ENDPOINT_URL']="http://localhost:8080"

# Login AWS
aws_profile = "vrt-analytics-engineer-nonsensitive"
envvars = subprocess.check_output(['aws-vault', 'exec', aws_profile, '--', 'env'])
for envline in envvars.split(b'\n'):
    line = envline.decode('utf8')
    eqpos = line.find('=')
    if eqpos < 4:
        continue
    k = line[0:eqpos]
    v = line[eqpos+1:]
    if k == 'AWS_ACCESS_KEY_ID':
        aws_access_key_id = v
    if k == 'AWS_SECRET_ACCESS_KEY':
        aws_secret_access_key = v
    if k == 'AWS_SESSION_TOKEN':
        aws_session_token = v

session = boto3.Session(
aws_access_key_id, aws_secret_access_key, aws_session_token,region_name="eu-west-1"
)
credentials = session.get_credentials()
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)
auth = AWSV4SignerAuth(credentials, 'eu-west-2', 'aoss')


environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
environ["AWS_SESSION_TOKEN"] = aws_session_token
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
mlflow.set_experiment("RAG")


with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        python_model=model,
        artifact_path="rag-model",
        pip_requirements=["-r requirements.txt"],
    )
    print(model_info.model_uri)

2024/09/05 11:22:22 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/05 11:22:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-steed-948 at: http://localhost:8080/#/experiments/732331809902987751/runs/4fed74c0d6694d9d84c817eaa6db5953.
2024/09/05 11:22:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/732331809902987751.


runs:/4fed74c0d6694d9d84c817eaa6db5953/rag-model


In [10]:
# Test predict
loaded_model = mlflow.pyfunc.load_model(
    model_uri=model_info.model_uri
)

/home/willem/miniconda3/envs/llm/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [17]:
mlflow.set_experiment("RAG")

@mlflow.trace
def predict(question):
    t = loaded_model.predict(question)
    return t["answer"]

In [18]:
predict("iets over geld en beleggen")

'Ik zou je graag de volgende programma\'s aanbevelen die relevant zijn voor je vraag over geld en beleggen:\n\n1. "Universiteit van Vlaanderen" - In deze episode wordt uitgelegd hoe bitcoins werken, hun nut en de gevolgen van virtueel geld. De episode is ideaal voor zowel beginners als gevorderden. Je kunt de episode vinden op deze [link](https://www.vrt.be/vrtnu/a-z/universiteit-van-vlaanderen/2017-2018/universiteit-van-vlaanderen-d20180425-s2017a93/).\n\n2. "Schermtijd" - In deze episode leert Margaux Bogaert hoe je veilig kunt beleggen met een app. De episode is beschikbaar op deze [link](https://www.vrt.be/vrtmax/a-z/schermtijd/1/schermtijd-s1a20/).\n\nDeze programma\'s bieden inzicht in verschillende aspecten van geld en beleggen, en zullen je helpen om je kennis op dit gebied te verbeteren.'